This notebook simulates data for a hypothetical farmland appraisal modeling task. We look in particular at the different model fits during k-fold cross-validation, the estimate of generalization error produced by cross-validation, and the confidence interval for the generalization error.

This notebook produces figures published in The Crosstab Kite's article [Research Digest: What does cross-validation really estimate?](https://crosstab.io/articles/bates-cross-validation), which digests the research paper [Cross-validation: what does it estimate and how well does it do it?](https://arxiv.org/abs/2104.00673) by Bates, Hastie, and Tibshirani.

The plot styling is intended for the figures as they appear in the article, so they look really bad in this notebook. That's known and ok.

# 0. Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

import scipy.stats as stats

pyo.init_notebook_mode()

In [ ]:
## Generic plot style
baseline_style = dict(
    font=dict(family="Arial", size=36),
    template="simple_white",
)

marker_size = 26

# 1. Generate data

The true regression function of sale price is quadratic in property acreage. The distribution of acreage and sale prices is intended to very loosely mimic agricultural property values in the Hill Country of Texas, based on [data from Texas A&M](https://www.recenter.tamu.edu/data/rural-land/).

In [ ]:
np.random.seed(18)

n = 100

acreage_mean = 120
acreage_sd = 30

price_sd = 350000
target = "price"

In [ ]:
df = pd.DataFrame({"acres": np.random.normal(acreage_mean, acreage_sd, n)})

noise = np.random.normal(loc=0, scale=price_sd, size=n)
df["sq_acres"] = df["acres"] ** 2
df[target] = 2000 * df["acres"] + 50 * df["sq_acres"] + noise
df.sample(5)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["acres"],
        y=df["price"],
        mode="markers",
        marker=dict(
            symbol="circle",
            color="rgba(100, 149, 237, 0.35)",
            size=marker_size,
            line=dict(width=2, color="#15388d"),
        ),
        showlegend=False,
    )
)

fig.update_layout(baseline_style)
fig.update_layout(xaxis_title="Acres", yaxis_title="Sale price ($)")
fig.write_image("sim_farm_sales.png", height=1400, width=1400)
fig.show()

Make a grid of values for the `acres` features, for plotting quadratic model fits.

In [ ]:
xgrid = pd.DataFrame(
    {"acres": np.linspace(df["acres"].min() - 5, df["acres"].max() + 5, 100)}
)
xgrid["sq_acres"] = xgrid["acres"] ** 2

# 2. Select the best model form with 5-fold cross-validation

Scikit-learn has a convenience function [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) that makes this a lot less verbose. Here we use the `KFold` iterator to show the steps more carefully and to more closely match the Bates, et al. paper. Specifically, the Bates, et al. paper computes the cross-validation error a little differently than most. Most sources say to take the average of the per-fold test errors, but Bates et al. record then error for each point when it's in the test, then take the average of all points at the end.

In [ ]:
linear_errors = np.array([])
quad_errors = np.array([])

kfold = KFold(n_splits=5)
for ix_train, ix_test in kfold.split(df):

    # Split data
    df_train = df.loc[ix_train]
    df_test = df.loc[ix_test]

    # Fit the linear model and get test RMSE
    linear_model = LinearRegression()
    linear_model.fit(df_train[["acres"]], df_train[[target]])
    linear_ystar = linear_model.predict(df_test[["acres"]]).flatten()
    linear_errors = np.append(linear_errors, (df_test[target] - linear_ystar))

    # Draw the trained linear model on the plot.
    fig.add_trace(
        go.Scatter(
            x=xgrid["acres"],
            y=linear_model.predict(xgrid[["acres"]]).flatten(),
            mode="lines",
            line=dict(width=3, dash="dash", color="orange"),
            showlegend=False,
        )
    )

    # Fit the quadratic model and get test RMSE
    quad_model = LinearRegression()
    quad_model.fit(df_train[["acres", "sq_acres"]], df_train[target])
    quad_ystar = quad_model.predict(df_test[["acres", "sq_acres"]]).flatten()
    quad_errors = np.append(quad_errors, (df_test[target] - quad_ystar))

    # Draw the trained quadratic model on the plot.
    fig.add_trace(
        go.Scatter(
            x=xgrid["acres"],
            y=quad_model.predict(xgrid[["acres", "sq_acres"]]).flatten(),
            mode="lines",
            line=dict(width=3, dash="dash", color="purple"),
            showlegend=False,
        )
    )

linear_cv_rmse = (linear_errors ** 2).mean() ** 0.5
quad_cv_rmse = (quad_errors ** 2).mean() ** 0.5

print(f"{linear_cv_rmse=}")
print(f"{quad_cv_rmse=}")

As expected, given that the true regression function is quadratic, the quadratic form has lower cross-validation error.

In [ ]:
fig.add_annotation(
    x=205,
    y=1.65e6,
    text=f"Linear model<br>5-fold CV fits<br>CV RMSE: ${linear_cv_rmse:,.2f}",
    showarrow=False,
    font=dict(color="orange"),
)

fig.add_annotation(
    x=150,
    y=2.8e6,
    text=f"Quadratic model<br>5-fold CV fits<br>CV RMSE: ${quad_cv_rmse:,.2f}",
    showarrow=False,
    font=dict(color="purple"),
)

fig.write_image("cv_model_fits.png", height=1400, width=1400)
fig.show()

# 3. Re-fit best predictive model to the full dataset

In [ ]:
final_model = LinearRegression()
final_model.fit(df[["acres", "sq_acres"]], df[target])

# 4. Illustrate generalization

What we really care about is the model's generalization error, which is the average model prediction error (measured by our squared error loss function) on new data points from the same distribution. Here we just manually create two new data points for the purpose of illustration on our schematic plot.

In [ ]:
df_new = pd.DataFrame({"acres": [90, 170]})
df_new["sq_acres"] = df_new["acres"] ** 2
df_new["ystar"] = final_model.predict(df_new[["acres", "sq_acres"]])
df_new["price"] = [5.8e5, 1.1e6]
df_new

Plot the final model with the new points and the model's predictions for those points.

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["acres"],
        y=df["price"],
        mode="markers",
        marker=dict(
            symbol="circle",
            color="rgba(100, 149, 237, 0.35)",
            size=marker_size,
            line=dict(width=2, color="#15388d"),
        ),
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        name="Final model",
        x=xgrid["acres"],
        y=final_model.predict(xgrid[["acres", "sq_acres"]]).flatten(),
        mode="lines",
        line=dict(width=6, color="purple"),
    )
)

fig.add_trace(
    go.Scatter(
        name="New point true values (unknown)",
        x=df_new["acres"],
        y=df_new["price"],
        mode="markers",
        marker=dict(
            symbol="circle-open", color="red", size=marker_size + 4, line_width=4
        ),
    )
)

fig.add_trace(
    go.Scatter(
        name="New point predictions",
        x=df_new["acres"],
        y=df_new["ystar"],
        mode="markers",
        marker=dict(symbol="x", color="red", size=marker_size + 4),
    )
)

fig.add_annotation(
    x=200,
    y=6.8e5,
    text="Averge RMSE<br>for new points: ?",
    showarrow=False,
    font=dict(color="red"),
)

fig.update_layout(baseline_style)
fig.update_layout(
    xaxis_title="Acres", yaxis_title="Sale price ($)", legend=dict(x=0.1, y=0.9)
)

fig.write_image("final_model.png", height=1400, width=1400)
fig.show()

# Naïve standard error and confidence interval of generalization error

This is what Bates, et al. call the *naïve cross-validation interval*. As they show, this is not a good idea - the interval is too narrow to cover the true generalization error with the intended frequency.

Note that even though our loss function is squared error, we take the square root here to get RMSE for interpretability.

In [ ]:
significance = 0.1
tail_prob = 1 - significance / 2
z_quantile = stats.norm.ppf(tail_prob)
print(f"{z_quantile=}")  # just write the value explicitly in the article

In [ ]:
std_err = (quad_errors ** 2).std(ddof=1) / np.sqrt(n)

avg_loss = quad_cv_rmse ** 2
rmse_ci_lower = (avg_loss - z_quantile * std_err) ** 0.5
rmse_ci_upper = (avg_loss + z_quantile * std_err) ** 0.5

print(f"{quad_cv_rmse=}")
print(f"{rmse_ci_lower=}")
print(f"{rmse_ci_upper=}")

This is a suprisingly high 90% confidence interval for generalization error. Crazy to think that it's not even wide enough to actually cover with 90% frequency.